In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv('hlth_cd_aro.csv')

In [ ]:
# Convert all columns to lowercase
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [ ]:
df.head()

In [ ]:
df.drop(columns=['dataflow', 'last_update', 'freq', 'unit', 'sex', 'age', 'obs_flag', 'resid'], inplace=True)

In [ ]:
# Drop geo too
df.drop(columns=['geo'], inplace=True)

In [ ]:
# Drop 2019 and 2020 as we are using absolute values and Uk is missing after 2018
filter = (df['time_period'] == 2019) | (df['time_period'] == 2020)
df.drop(index=df[filter].index, inplace=True)

In [ ]:
# Replace all NaN with 0
df.fillna(0, inplace=True)
df.head()

In [ ]:
df.dtypes

In [ ]:
all = df.groupby(['icd10', 'time_period']).sum().reset_index()
all.head()

In [ ]:
top2012 = all[all['time_period'] == 2012].sort_values(by='obs_value', ascending=False).reset_index(drop=True)
top2012_list = top2012['icd10'].tolist()[1:20]

In [ ]:
top2018 = all[all['time_period'] == 2018].sort_values(by='obs_value', ascending=False).reset_index(drop=True)
top2018_list = top2018['icd10'].tolist()[1:20]

In [ ]:
# Merge the lists and keep only unique names
merged = set(top2012_list) | set(top2018_list)

# Convert the set back to a list
unique_diseases = list(merged)
# print(unique_diseases)

In [ ]:
# group the data that matches the filters and add up the values
grouped_data = df.loc[df['icd10'].isin(unique_diseases)].groupby(['icd10', 'time_period']).sum().reset_index()
other_data = df.loc[~df['icd10'].isin(unique_diseases)].groupby(['time_period'])['obs_value'].sum().reset_index()

In [ ]:
other_data['icd10'] = 'Others'

In [ ]:
# concatenate the two DataFrames
top = pd.concat([grouped_data, other_data], sort=False)

In [ ]:
# tot = top.groupby('icd10')['obs_value'].sum().sort_values(ascending=False)

In [ ]:
# other_data.pivot(index='icd10', columns='time_period', values='obs_value')

In [ ]:
sankey = top.pivot(index='icd10', columns='time_period', values='obs_value').reset_index()

In [ ]:
sankey.to_csv('death_causes.csv', index=False)

In [ ]:
# Disease = 'Diseases of the circulatory system (I00-I99)'
# test = all[all['icd10'] == Disease]
# test.head()

In [ ]:
# fig, axs = plt.subplots(figsize=(12, 3))
# axs.bar(test['time_period'], test['obs_value'])
# plt.title(Disease)
# plt.xlabel('Year')
# plt.ylabel('Tot')
# plt.yscale('log')
# plt.show()